In [1]:
import pandas as pd
import numpy as np
import bitcoin
import datetime
from ping3 import ping
import random
import json
import re
import multiprocess as mp

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/data'

In [13]:
with open("./website.json", "r") as f:
    website_list = json.load(f)

website_list[:10]

[{'id': '2205',
  'name': '人人网',
  'logo': 'http://reg007.u.qiniudn.com/app/renren.jpg',
  'desc': '人人网，中国领先的实名制SNS社交网络。加入人人网，找到老同学，结识新朋友。',
  'category': '社交',
  'home': 'http://www.renren.com/'},
 {'id': '2217',
  'name': '途牛旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/tuniu.jpg',
  'desc': '途牛旅游网-中国专业全面的旅游网,客户满意度94%;提供自助游(自由行),国内旅游,出境旅游,自驾游,公司旅游。低价保证,专业服务,九大出游保障,服务超百万人出游。',
  'category': '旅行',
  'home': 'http://www.tuniu.com/'},
 {'id': '2224',
  'name': '酷讯旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/kuxun.jpg',
  'desc': '酷讯旅游网(Kuxun.cn)是中国领先的在线旅游媒体。为您提供低价可靠的国内国际机票，特价机票99元起，另提供全国20万家酒店预订查询、酒店评论，最低2折起，以及列车时刻表、50000多旅游景点的旅游攻略信息，为您的度假提供最佳的旅游线路。',
  'category': '旅行',
  'home': 'http://www.kuxun.cn/'},
 {'id': '2246',
  'name': '金山词霸',
  'logo': 'http://reg007.u.qiniudn.com/app/iciba.jpg',
  'desc': '爱词霸英语为广大网友提供在线翻译、在线词典、英语口语、英语学习资料、汉语词典，金山词霸下载等服务，致力于为您提供优质权威的在线英语服务，是5000万英语学习者的首选。',
  'category': '工具',
  'home': 'http://www.iciba.com/'},
 {'id': '2247',
  'name': '一听音乐网',
  '

In [14]:
pattern = re.compile("http://(.+?)/")
url_list = [re.findall(pattern, dic["home"])[0] for dic in website_list]

url_list[:10]

['www.renren.com',
 'www.tuniu.com',
 'www.kuxun.cn',
 'www.iciba.com',
 'www.1ting.com',
 'www.youku.com',
 'www.zol.com.cn',
 'www.baihe.com',
 'www.izhenxin.com',
 'www.taobao.com']

In [15]:
url = random.choice(url_list)
delay = ping(url)

url
delay

'www.meituan.com'

0.04405498504638672

In [16]:
def gen_addr():
    # 生成一个随机的密钥
    while True:
        # 生成一个用十六进制表示的长 256 位的私钥（str类型）
        private_key = bitcoin.random_key()
        # 解码为十进制的整形密钥
        decoded_private_key = bitcoin.decode_privkey(private_key, "hex")
        if 0 < decoded_private_key < bitcoin.N:
            break
    # print(private_key) #密钥（十六进制,长 256 位）

    # 计算地址
    # 用 WIF 格式编码密钥
    wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, "wif")
    # 用 01 标识的压缩密钥
    compressed_private_key = private_key + "01"
    # 生成 WIF的压缩格式
    wif_compressed_private_key = bitcoin.encode_privkey(
        bitcoin.decode_privkey(compressed_private_key, "hex"), "wif"
    )
    # 计算公钥坐标 K = k * G
    public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
    # 计算公钥
    hex_encoded_public_key = bitcoin.encode_pubkey(public_key, "hex")
    # 计算压缩公钥
    # if public_key[1] % 2 == 0:  # 两种方式均可
    if public_key[1] & 1 == 0:
        compressed_prefix = "02"
    else:
        compressed_prefix = "03"
    # 转十六也可用 bitcoin.encode(xxx, 16)
    hex_compressed_public_key = compressed_prefix + hex(public_key[0])[2:]
    # print(f'压缩公钥（十六进制）{hex_compressed_public_key} '
    #      '（02 开头代表 y 是偶数，03 开头代表 y 是奇数）')
    # 传入公钥坐标对象/十六进制公钥值，输出同样的地址
    address = bitcoin.pubkey_to_address(public_key)
    # print(address) #地址（b58check）（1 开头
    return address

In [19]:
df_user = pd.DataFrame(columns=["user_id", "address", "url"])
for i in range(2000):
    url = random.choice(url_list)
    # while not ping(url):
    #     url=random.choice(url_list)
    df_user.loc[i] = [i, gen_addr(), url]

df_user

,user_id,address,url
0,0,1LUjEfWLZhrndJBkkqXABxpaQNQskBZGqY,www.elong.com
1,1,17Bo6VWMnfks9Amz35MKzKsBZqzBdU7P8m,www.youku.com
2,2,1DVBvtVz8Gj61Sjvea61TP3vD46xx7ApgB,www.tuan800.com
3,3,1Gc2G3fHt41uFWbmfLSwLuevXx72m36HGB,www.xianguo.com
4,4,1JKifazQtMr4F4hBQhBF4Pc2M6cPRkXEn7,www.suning.com
...,...,...,...
1995,1995,161WhqU8FV7ZL32S7fBRo6kQzTfosg4kHN,www.aibang.com
1996,1996,16E1hPw1UGy1uF6ASbTwG2doH1j13sLP17,bbs.dedecms.com
1997,1997,19F59LeELN4d3hrKoHccq42Cf3Wa1qYxHD,www.100.com
1998,1998,1Fm8iVjo2ioUDDRP7y1tjS6Cug7AmfDe3E,www.weibo.com


In [20]:
df_user.to_csv("user.csv", index=False)

---

In [26]:
!python gen_tx.py;

Finished round 1
Process Process-952:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/py38/lib/python3.8/multiprocessing/managers.py", line 827, in _callmethod
    conn = self._tls.connection
AttributeError: 'ForkAwareLocal' object has no attribute 'connection'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/py38/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/py38/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/data/gen_tx.py", line 34, in gen_tx
    tx_list.append([send_ts, recv_ts, latency, label, from_user, to_user, value])
  File "<string>", line 2, in append
  File "/opt/homebrew/Caskroom/miniforge/base/envs/py38/lib/python3.8/multipr

In [3]:
df_tx = pd.read_pickle("./tx_9969.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-10-12 11:28:21.816994,2022-10-12 11:28:21.834958,0.017964,0,1D59QhANFNFCVxXmcJKhUD5VjDoNBSBxGc,161WhqU8FV7ZL32S7fBRo6kQzTfosg4kHN,3.969223
1,2022-10-12 11:28:21.841826,2022-10-12 11:28:21.888259,0.046433,0,1B7RMak7rSPdxSmE3eUe4qEgsbbc86XAzq,1BAtUeRc2t1935tDjnMhKJ4CTT3ERXjF5y,8.002439
2,2022-10-12 11:28:21.878489,2022-10-12 11:28:21.900837,0.022348,0,1FvWp1BZCWNWbN9P5BwMQBCQRJrCiyEf34,1CyRSXpmjVSXsWBcAaTyV1sUqB2RbqEGo5,3.842746
3,2022-10-12 11:28:21.937969,2022-10-12 11:28:21.961715,0.023746,0,16WoGrb6D65KBjodhBFc2fLYaMnn4aNsGs,19Y3gSmczyJkFJ7hTbL3sYNxjKfiudAMGx,5.572624
4,2022-10-12 11:28:21.911149,2022-10-12 11:28:21.980410,0.069261,1,1PnR481Zz6u3MTdtMJnXH8YziHVcd7JcDJ,12M23XoTqEK1A2F5UPUydpHBWMY6TrsnZd,1.681064
...,...,...,...,...,...,...,...
9964,2022-10-12 11:37:07.846933,2022-10-12 11:37:07.961303,0.114370,1,1Jw9BHSzEtzVff7qjxxSnyt8JPqN8ko4C4,1GjJ91SYSmeAyCgvjzmA4UBbaWcjAt7XBj,1.762662
9965,2022-10-12 11:37:07.965230,2022-10-12 11:37:07.999769,0.034539,0,1Faghv74NTSrAtV9kKCkuZz8vLUDU9NyYP,1M8SmdNzt1gzfD34nB9KHDP7j5YzgP8VbR,7.744528
9966,2022-10-12 11:37:07.987328,2022-10-12 11:37:08.008256,0.020928,0,1HbSAKyZPuSPdPF44g5esNP7iLqfyDJ1cd,1rNiiQKw5UMbnEpqsXKkU5s9CeWh1qJ2p,4.029260
9967,2022-10-12 11:37:08.011687,2022-10-12 11:37:08.047103,0.035416,0,12bgNUZ8wKetxSBNLTxFDPt4fj52S5bREu,199GuGHpDyG8xrg49XxjLVLixCmziCdCcD,4.795328


In [4]:
sum(df_tx["label"] == 1)

1783

---

---

Generate training data

In [10]:
df_tx = pd.read_pickle("./tx_9969.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-10-12 11:28:21.816994,2022-10-12 11:28:21.834958,0.017964,0,1D59QhANFNFCVxXmcJKhUD5VjDoNBSBxGc,161WhqU8FV7ZL32S7fBRo6kQzTfosg4kHN,3.969223
1,2022-10-12 11:28:21.841826,2022-10-12 11:28:21.888259,0.046433,0,1B7RMak7rSPdxSmE3eUe4qEgsbbc86XAzq,1BAtUeRc2t1935tDjnMhKJ4CTT3ERXjF5y,8.002439
2,2022-10-12 11:28:21.878489,2022-10-12 11:28:21.900837,0.022348,0,1FvWp1BZCWNWbN9P5BwMQBCQRJrCiyEf34,1CyRSXpmjVSXsWBcAaTyV1sUqB2RbqEGo5,3.842746
3,2022-10-12 11:28:21.937969,2022-10-12 11:28:21.961715,0.023746,0,16WoGrb6D65KBjodhBFc2fLYaMnn4aNsGs,19Y3gSmczyJkFJ7hTbL3sYNxjKfiudAMGx,5.572624
4,2022-10-12 11:28:21.911149,2022-10-12 11:28:21.980410,0.069261,1,1PnR481Zz6u3MTdtMJnXH8YziHVcd7JcDJ,12M23XoTqEK1A2F5UPUydpHBWMY6TrsnZd,1.681064
...,...,...,...,...,...,...,...
9964,2022-10-12 11:37:07.846933,2022-10-12 11:37:07.961303,0.114370,1,1Jw9BHSzEtzVff7qjxxSnyt8JPqN8ko4C4,1GjJ91SYSmeAyCgvjzmA4UBbaWcjAt7XBj,1.762662
9965,2022-10-12 11:37:07.965230,2022-10-12 11:37:07.999769,0.034539,0,1Faghv74NTSrAtV9kKCkuZz8vLUDU9NyYP,1M8SmdNzt1gzfD34nB9KHDP7j5YzgP8VbR,7.744528
9966,2022-10-12 11:37:07.987328,2022-10-12 11:37:08.008256,0.020928,0,1HbSAKyZPuSPdPF44g5esNP7iLqfyDJ1cd,1rNiiQKw5UMbnEpqsXKkU5s9CeWh1qJ2p,4.029260
9967,2022-10-12 11:37:08.011687,2022-10-12 11:37:08.047103,0.035416,0,12bgNUZ8wKetxSBNLTxFDPt4fj52S5bREu,199GuGHpDyG8xrg49XxjLVLixCmziCdCcD,4.795328


In [16]:
df_user=pd.read_csv("./user.csv")

user_list=df_user["address"].values.tolist()

user_list[:5]

['1LUjEfWLZhrndJBkkqXABxpaQNQskBZGqY',
 '17Bo6VWMnfks9Amz35MKzKsBZqzBdU7P8m',
 '1DVBvtVz8Gj61Sjvea61TP3vD46xx7ApgB',
 '1Gc2G3fHt41uFWbmfLSwLuevXx72m36HGB',
 '1JKifazQtMr4F4hBQhBF4Pc2M6cPRkXEn7']

In [17]:
user_list.index("17Bo6VWMnfks9Amz35MKzKsBZqzBdU7P8m")

1

In [18]:
def get_user_id(row):
    from_user_id=user_list.index(row["from"])
    to_user_id=user_list.index(row["to"])
    
    row["from_user_id"]=from_user_id
    row["to_user_id"]=to_user_id
    
    return row

In [32]:
df_data=df_tx.copy()
df_data=df_data.apply(get_user_id, axis=1)

df_data

,send_timestamp,recv_timestamp,latency,label,from,to,value,from_user_id,to_user_id
0,2022-10-12 11:28:21.816994,2022-10-12 11:28:21.834958,0.017964,0,1D59QhANFNFCVxXmcJKhUD5VjDoNBSBxGc,161WhqU8FV7ZL32S7fBRo6kQzTfosg4kHN,3.969223,49,1995
1,2022-10-12 11:28:21.841826,2022-10-12 11:28:21.888259,0.046433,0,1B7RMak7rSPdxSmE3eUe4qEgsbbc86XAzq,1BAtUeRc2t1935tDjnMhKJ4CTT3ERXjF5y,8.002439,735,976
2,2022-10-12 11:28:21.878489,2022-10-12 11:28:21.900837,0.022348,0,1FvWp1BZCWNWbN9P5BwMQBCQRJrCiyEf34,1CyRSXpmjVSXsWBcAaTyV1sUqB2RbqEGo5,3.842746,443,1530
3,2022-10-12 11:28:21.937969,2022-10-12 11:28:21.961715,0.023746,0,16WoGrb6D65KBjodhBFc2fLYaMnn4aNsGs,19Y3gSmczyJkFJ7hTbL3sYNxjKfiudAMGx,5.572624,1076,922
4,2022-10-12 11:28:21.911149,2022-10-12 11:28:21.980410,0.069261,1,1PnR481Zz6u3MTdtMJnXH8YziHVcd7JcDJ,12M23XoTqEK1A2F5UPUydpHBWMY6TrsnZd,1.681064,10,1389
...,...,...,...,...,...,...,...,...,...
9964,2022-10-12 11:37:07.846933,2022-10-12 11:37:07.961303,0.114370,1,1Jw9BHSzEtzVff7qjxxSnyt8JPqN8ko4C4,1GjJ91SYSmeAyCgvjzmA4UBbaWcjAt7XBj,1.762662,1667,708
9965,2022-10-12 11:37:07.965230,2022-10-12 11:37:07.999769,0.034539,0,1Faghv74NTSrAtV9kKCkuZz8vLUDU9NyYP,1M8SmdNzt1gzfD34nB9KHDP7j5YzgP8VbR,7.744528,1363,774
9966,2022-10-12 11:37:07.987328,2022-10-12 11:37:08.008256,0.020928,0,1HbSAKyZPuSPdPF44g5esNP7iLqfyDJ1cd,1rNiiQKw5UMbnEpqsXKkU5s9CeWh1qJ2p,4.029260,786,994
9967,2022-10-12 11:37:08.011687,2022-10-12 11:37:08.047103,0.035416,0,12bgNUZ8wKetxSBNLTxFDPt4fj52S5bREu,199GuGHpDyG8xrg49XxjLVLixCmziCdCcD,4.795328,177,1503


In [37]:
df_data.drop(columns=["send_timestamp", "latency", "from", "to"], inplace=True)

df_data

,recv_timestamp,label,value,from_user_id,to_user_id
0,2022-10-12 11:28:21.834958,0,3.969223,49,1995
1,2022-10-12 11:28:21.888259,0,8.002439,735,976
2,2022-10-12 11:28:21.900837,0,3.842746,443,1530
3,2022-10-12 11:28:21.961715,0,5.572624,1076,922
4,2022-10-12 11:28:21.980410,1,1.681064,10,1389
...,...,...,...,...,...
9964,2022-10-12 11:37:07.961303,1,1.762662,1667,708
9965,2022-10-12 11:37:07.999769,0,7.744528,1363,774
9966,2022-10-12 11:37:08.008256,0,4.029260,786,994
9967,2022-10-12 11:37:08.047103,0,4.795328,177,1503


In [38]:
df_data["recv_timestamp"]=df_data["recv_timestamp"].apply(lambda x: x.value)

df_data

,recv_timestamp,label,value,from_user_id,to_user_id
0,1665574101834958000,0,3.969223,49,1995
1,1665574101888259000,0,8.002439,735,976
2,1665574101900837000,0,3.842746,443,1530
3,1665574101961715000,0,5.572624,1076,922
4,1665574101980410000,1,1.681064,10,1389
...,...,...,...,...,...
9964,1665574627961303000,1,1.762662,1667,708
9965,1665574627999769000,0,7.744528,1363,774
9966,1665574628008256000,0,4.029260,786,994
9967,1665574628047103000,0,4.795328,177,1503


In [39]:
df_data.to_pickle(f"./data_{len(df_data)}.pkl")